In [7]:
import transformers
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [2]:
df = pd.read_csv('data/processed/AITA-Reddit-Dataset.csv')

In [3]:
df

,text,ESH,NAH,NTA,YTA
0,i turned 30 on monday. i don't have a problem ...,0.0,0.0,1.0,0.0
1,feed up with my gf. for 6 years ive supported ...,0.0,0.0,1.0,0.0
2,so at the start of the school year. like any ...,0.0,0.0,1.0,0.0
3,"sorry for format, using mobile. so a little bi...",0.0,0.0,1.0,0.0
4,so next week is finals week at my university a...,0.0,0.0,1.0,0.0
...,...,...,...,...,...
270704,"my great grandma gigi, ran a business back in ...",0.0,0.0,1.0,0.0
270705,so my (15f) birthday was a few days ago and my...,0.0,0.0,1.0,0.0
270706,i have already posted this story but now i’m p...,0.0,0.0,1.0,0.0
270707,"character line-up: me (32f), and my girlfriend...",0.0,0.0,0.0,1.0


In [4]:
model_name = 'bert-base-uncased'

model = transformers.BertForSequenceClassification.from_pretrained(model_name, num_labels=4)
tokenizer = transformers.BertTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
classifier = transformers.pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

2023-11-25 16:13:25.129867: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-25 16:13:25.129899: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-25 16:13:25.129935: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-25 16:13:25.136775: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-25 16:13:25.873882: W tensorflow/compiler/

In [9]:
train_dataset = df.iloc[:int(len(df)*0.8)]
eval_dataset = df.iloc[int(len(df)*0.8):]

In [8]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return f1

In [13]:
# finetune the model
training_args = transformers.TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=4,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=500,
)
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics    
)
trainer.train()
    

  0%|          | 0/162426 [00:00<?, ?it/s]

KeyError: 203581